In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [27]:
# Load the dataset into a Pandas DataFrame
df1 = pd.read_csv("C:/Users/vrl2k/Desktop/MS/Spring 2023/Deep Learning/Project/MIMIC db/DIAGNOSES_ICD.csv")

In [28]:
# Remove Duplicate and group by SUBJECT_ID

unique_combinations = df1.groupby(['SUBJECT_ID', 'ICD9_CODE']).size().reset_index().rename(columns={0: 'count'}).nunique()

print(unique_combinations)

SUBJECT_ID    46517
ICD9_CODE      6984
count            32
dtype: int64


In [29]:
#Import ICD lookup file 

df_icd = pd.read_csv("C:/Users/vrl2k/Desktop/MS/Spring 2023/Deep Learning/Project/MIMIC db/D_ICD_DIAGNOSES.csv")

In [30]:
# Join patient_records with ICD9_CODE to add ICD short_title and long_title 

merged_df2 = pd.merge(df1, df_icd, on='ICD9_CODE')

In [31]:
# Drop unnecessary fields 

merged_df2 = merged_df2.drop(['ROW_ID_y'], axis=1)

In [32]:
# remove duplicate rows

df_dedup = merged_df2.drop_duplicates()

In [33]:
# replace all special characters with space in the 'text' column
df_dedup['LONG_TITLE'] = df_dedup['LONG_TITLE'].replace('[^a-zA-Z0-9]', ' ', regex=True)
df_dedup['SHORT_TITLE'] = df_dedup['SHORT_TITLE'].replace('[^a-zA-Z0-9]', ' ', regex=True)

In [34]:
# remove numbers from column TEXT
df_dedup['LONG_TITLE'] = df_dedup['LONG_TITLE'].replace({'\d+': ''}, regex=True)
df_dedup['SHORT_TITLE'] = df_dedup['SHORT_TITLE'].replace({'\d+': ''}, regex=True)

In [35]:
# replace numbers with space
df_dedup = df_dedup.replace('\d+', ' ', regex=True)

In [36]:
df_dedup.LONG_TITLE = df_dedup.LONG_TITLE.str.replace('\n', '')  # remove newline
df_dedup.LONG_TITLE = df_dedup.LONG_TITLE.str.replace('\r', '')  # carriage returns

df_dedup.SHORT_TITLE = df_dedup.SHORT_TITLE.str.replace('\n', '')  # remove newline
df_dedup.SHORT_TITLE = df_dedup.SHORT_TITLE.str.replace('\r', '')  # carriage returns

In [37]:
df_dedup.LONG_TITLE = df_dedup.LONG_TITLE.str.lower()  # convert to lower case
df_dedup.SHORT_TITLE = df_dedup.SHORT_TITLE.str.lower()  # convert to lower case

In [39]:
df_clean = df_dedup 

In [41]:
df_clean.head(5)

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,1297,109,172335,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
1,1311,109,173633,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
2,1019,109,131345,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
3,1039,109,131376,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
4,1059,109,135923,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...


In [43]:
df_clean.shape

(634709, 7)

In [45]:
# Select only recoreds with SUBJECT_ID record count 5 or less. 

counts = df_clean['SUBJECT_ID'].value_counts()
df_filtered = df_clean[df_clean['SUBJECT_ID'].isin(counts[counts >= 5].index)]

In [47]:
df_filtered.head(5)

,ROW_ID_x,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,1297,109,172335,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
1,1311,109,173633,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
2,1019,109,131345,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
3,1039,109,131376,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...
4,1059,109,135923,1.0,,mal hyp kid w cr kid v,hypertensive chronic kidney disease malignant...


In [48]:
df_filtered.shape

(612776, 7)

In [49]:
df_filtered.to_csv('C:/Users/vrl2k/Desktop/MS/Spring 2023/Deep Learning/Project/Proj_mimic_out/patient_cleanfile.csv', index=False)

In [50]:
df_filtered.nunique()

ROW_ID_x       612776
SUBJECT_ID      39760
HADM_ID         52074
SEQ_NUM            39
ICD9_CODE           3
SHORT_TITLE      6630
LONG_TITLE       6714
dtype: int64